# Production Runbook

This notebook mirrors the production steps with checkpoints and expected artifacts.

In [ ]:
# Environment sanity
import sys, subprocess
print('Python:', sys.executable)
subprocess.run(['pip', 'install', '-e', '.'], check=True)


## 1) Data ingestion and features

In [ ]:
import subprocess

subprocess.run(['python', '-m', 'gridpulse.data_pipeline.validate_schema', '--in', 'data/raw', '--report', 'reports/data_quality_report.md'], check=True)
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.build_features', '--in', 'data/raw', '--out', 'data/processed'], check=True)
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.split_time_series', '--in', 'data/processed/features.parquet', '--out', 'data/processed/splits'], check=True)


## 2) Baselines + GBM

In [ ]:
import subprocess

for target in ['load_mw', 'wind_mw', 'solar_mw']:
    subprocess.run(['python', '-m', 'gridpulse.forecasting.train_baseline', '--features', 'data/processed/features.parquet', '--splits', 'data/processed/splits', '--target', target], check=True)


## 3) Deep models (LSTM + TCN)

In [ ]:
import subprocess

subprocess.run(['python', '-m', 'gridpulse.forecasting.train', '--config', 'configs/train_forecast.yaml'], check=True)


## 4) Start API + Dashboard (run in terminals)

- `uvicorn services.api.main:app --reload --port 8000`
- `streamlit run services/dashboard/app.py`


## 5) Monitoring + Optimization

- `curl http://localhost:8000/monitor`
- `curl -X POST http://localhost:8000/optimize -H 'Content-Type: application/json' -d '{"forecast_load_mw":[8000,8200],"forecast_renewables_mw":[3200,3100]}'`
